In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
segments = pd.read_csv("segments_final.csv")

In [ ]:
#determines if each segment on the map is walkable or not
for i in range(len(segments)):
  value = segments.iloc[i, -1]
  if pd.isna(value) or (isinstance(value, str) and '"bicycle"=>"dismount"' in value) or (isinstance(value, str) and '"ramp"=>"no"' in value):
      segments.loc[i, 'walkable'] = 1 #1 is walk
  else:
      segments.loc[i, 'walkable'] = 0 #0 is bike

In [ ]:
#function to split lines into smaller segments
def fix_wkt(arr, walkable):
  if len(arr) == 5:
    final_segments.loc[len(final_segments)] = [arr[0] + ' ((' + arr[1] + ' ' + arr[2] + ',' + arr[3] + ' ' + arr[4] + '))', arr[1] + ' ' + arr[2], arr[3] + ' ' + arr[4], walkable]
  else:
    fix_wkt([arr[0], arr[-4], arr[-3], arr[-2], arr[-1]], walkable)
    fix_wkt(arr[0:-2], walkable)

In [ ]:
#runs function on segments dataframe
final_segments = pd.DataFrame(columns = ['WKT', 'start', 'end', 'walkable'])
for i in range(len(segments)):
  arr = segments.iloc[i, 0].replace(',', ' ').replace('(', ' ').replace(')', ' ').split()
  fix_wkt(arr, segments.iloc[i]['walkable'])

In [ ]:
#Build a dictionary `neighbors` where each key is a start point, and the value is a list of tuples
# containing the connected endpoint and whether the segment is walkable.
neighbors = dict()
for i in range(len(final_segments)):
  start = final_segments.iloc[i]['start']
  end = final_segments.iloc[i]['end']
  walkable = final_segments.iloc[i]['walkable']
  if neighbors.get(start) is None:
    neighbors[start] = []
  neighbors[start].append((end, walkable))
  if neighbors.get(end) is None:
    neighbors[end] = []
  neighbors[end].append((start, walkable))

In [ ]:
#import nodes csv and create coord column that joins x and y coordinates
nodes = pd.read_csv("nodes_fr.csv")
nodes["coord"] = nodes["X"].astype(str) + " " + nodes["Y"].astype(str)
nodes = nodes[["id", "coord"]]
nodes["result"] = "none"

In [ ]:
#manually add a missing node into a roundabout on the map to fix a cycle
nodes.loc[len(nodes)] = [len(nodes) + 1, '-118.4401695 34.0704353', 'none']

In [ ]:
#label node's ids
nodes['ID'] = None
for i in range(len(nodes)):
  nodes.at[i, 'ID'] = str(i + 1)

In [ ]:
#create results matrix that will store the paths between nodes
result_matrix = [['' for j in range(len(nodes))] for i in range(len(nodes))]

In [ ]:
#depth-first-search algorithm to find all possible paths between nodes
def message_pass(last_visted, coord, message):
  for neighbor in neighbors[coord]:
    #if neighbor has already been visited, skip to next neighbor
    if neighbor[0] == last_visted:
      continue

    #add coordinate of neighbor to message
    new_message = message + neighbor[0] + ", "

    #change walkability if needed
    if not message[0]:
      new_message[0] = neighbor[1]

    row_id, col_id = 0, 0
    if nodes["coord"].isin([neighbor[0]]).any():
      #if neighbor is a node, we have reached stopping point, so append neighbor id to end of message
      new_message = new_message + str(nodes.loc[nodes["coord"] == neighbor[0]]["ID"].values[0])

      #parses message to find the start and end nodes of path, the smaller id is the row index, larger id is the col index (matrix will be upper triangular)
      arr = new_message.replace(',', ' ').replace('(', ' ').replace(')', ' ').replace('|', ' ').split()
      row_id, col_id = min(int(arr[1]), int(arr[-1])), max(int(arr[1]), int(arr[-1]))
      if result_matrix[row_id-1][col_id-1] != '':y
        #if the path between two nodes is not empty, means has already been reached, so can move on
        continue
      else:
        #if empty, set the element in matrix to the message
        result_matrix[row_id-1][col_id-1] = new_message

        #now recurisvely call the function, now treating the neighbor as the starting coordinate
        message_pass("", neighbor[0], "0 | " + str(nodes.loc[nodes["coord"] == neighbor[0]]["ID"].values[0]) + ", " + neighbor[0] + ", ")
    else:
      #if neighbor is not a node, we just carry on recursively calling now starting at neighbor, but with no end node id yet
      message_pass(coord, neighbor[0], new_message)

message_pass("", "-118.4397642 34.0700517", "0 | 1, -118.4397642 34.0700517, ")

Streaming output truncated to the last 5000 lines.
16 427 is full
17 20
17 20 is full
from-118.4537867 34.0757012 to -118.4537363 34.07577
from-118.4537363 34.07577 to -118.4537227 34.0758445
from-118.4537227 34.0758445 to -118.4537387 34.0758946
from-118.4537387 34.0758946 to -118.4538681 34.0761151
from-118.4538681 34.0761151 to -118.4538891 34.0762478
from-118.4538891 34.0762478 to -118.4538608 34.0763498
from-118.4538608 34.0763498 to -118.4538028 34.076456
20 1289
20 1289 is blank
spawn: 20 1289
from-118.4537153 34.0765989 to -118.4538028 34.076456
from-118.4538028 34.076456 to -118.4538608 34.0763498
from-118.4538608 34.0763498 to -118.4538891 34.0762478
from-118.4538891 34.0762478 to -118.4538681 34.0761151
from-118.4538681 34.0761151 to -118.4537387 34.0758946
from-118.4537387 34.0758946 to -118.4537227 34.0758445
from-118.4537227 34.0758445 to -118.4537363 34.07577
20 1289
20 1289 is full
from-118.4537153 34.0765989 to -118.4536047 34.076655
from-118.4536481 34.0756051 to -118

In [ ]:
# test case to check if all nodes are reached
connected = [0]*len(nodes)
for i in range(len(result_matrix)):
  for j in range(len(result_matrix)):
    #if the element is not empty, that means these two nodes are reached
    if result_matrix[i][j] != "":
      connected[i] = 1
      connected[j] = 1
print(sum(connected))

1290


In [ ]:
#checks to see if network is fully connected
seg_simple = final_segments[["start", "end"]]

# Create an undirected graph
G = nx.Graph()

# Add edges to the graph based on the segments
for _, row in seg_simple.iterrows():
    G.add_edge(row['start'], row['end'])

# Check if all nodes are connected (i.e., there is a path between all nodes)
if nx.is_connected(G):
    print("All segments are interconnected!")
else:
    print("The segments are not all interconnected.")


num_components = nx.number_connected_components(G)
print(f"Number of connected components: {num_components}")

connected_components = list(nx.connected_components(G))
# Count the nodes in each component
component_sizes = [len(component) for component in connected_components]
print("Sizes of each connected component:", component_sizes)

All segments are interconnected!
Number of connected components: 1
Sizes of each connected component: [4250]


In [ ]:
#export result matrix
pd.DataFrame(result_matrix).to_csv('paths_matrix.csv', index = False)